<a href="https://colab.research.google.com/github/h120750572/Auto-Deep-Research/blob/main/EBM_write_report_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython python-docx reportlab

from Bio import Entrez
from docx import Document

# ✅ PubMed API 設定
Entrez.email = "lee.youngchang@gmail.com"
Entrez.api_key = "6b6183ed0ec2f5b77583277dd4f56070b508"

# -------- Step 1. Query -------- #
def generate_pico_queries(patient, intervention, comparison, outcomes):
    if isinstance(outcomes, list):
        outcome_str_pubmed = " OR ".join([f'"{o}"' for o in outcomes])
    else:
        outcome_str_pubmed = f'"{outcomes}"'
    return (
        f'("{patient}" AND "{intervention}" AND "{comparison}" AND ({outcome_str_pubmed})) '
        f'AND (RCT OR "randomized controlled trial" OR review OR cohort) '
        f'AND ("2020"[Date - Publication] : "3000"[Date - Publication])'
    )

# -------- Step 2. PubMed Fetch -------- #
def fetch_pubmed_articles(query, max_results=15):
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results, sort="relevance")
    record = Entrez.read(handle)
    handle.close()
    pmids = record["IdList"]

    results = []
    if pmids:
        handle = Entrez.efetch(db="pubmed", id=",".join(pmids), rettype="medline", retmode="xml")
        papers = Entrez.read(handle)
        handle.close()

        for paper in papers["PubmedArticle"]:
            try:
                title = paper["MedlineCitation"]["Article"]["ArticleTitle"]
                authors = paper["MedlineCitation"]["Article"]["AuthorList"]
                author_str = ", ".join(
                    [f"{a['LastName']} {a.get('Initials','')}" for a in authors if "LastName" in a]
                )
                journal = paper["MedlineCitation"]["Article"]["Journal"]["Title"]
                year = paper["MedlineCitation"]["Article"]["Journal"]["JournalIssue"]["PubDate"].get("Year", "n.d.")
                abstract = paper["MedlineCitation"]["Article"].get("Abstract", {}).get("AbstractText", [""])[0]
                doi = None
                for id in paper["PubmedData"]["ArticleIdList"]:
                    if id.attributes["IdType"] == "doi":
                        doi = str(id)
                results.append({
                    "title": title,
                    "authors": author_str,
                    "journal": journal,
                    "year": year,
                    "doi": doi,
                    "pmid": paper["MedlineCitation"]["PMID"],
                    "abstract": abstract
                })
            except Exception as e:
                print("Error parsing paper:", e)
    return results

# -------- Step 3. Critical Appraisal -------- #
def auto_jadad_score(abstract):
    score = 0
    text = abstract.lower()
    if "randomized" in text: score += 1
    if "computer" in text or "random number" in text or "table" in text: score += 1
    if "double blind" in text or "placebo" in text: score += 1
    if "identical" in text or "matching placebo" in text: score += 1
    if "withdrawal" in text or "lost to follow" in text or "dropout" in text: score += 1
    return min(score, 5)

def auto_nos_score(abstract):
    score = {"Selection": 0, "Comparability": 0, "Outcome": 0}
    text = abstract.lower()
    if "cohort" in text or "prospective" in text or "retrospective" in text: score["Selection"] = 1
    if "matched" in text or "adjusted" in text or "controlled" in text: score["Comparability"] = 1
    if "follow-up" in text or "mortality" in text or "pregnancy rate" in text or "outcome" in text: score["Outcome"] = 1
    return score, sum(score.values())

def auto_grade(abstract):
    text = abstract.lower()
    if "randomized" in text: return "High"
    elif "cohort" in text: return "Moderate"
    else: return "Low"

def auto_appraise(articles):
    appraisal = []
    for i, art in enumerate(articles, 1):
        abs_text = art['abstract'] if art['abstract'] else ""
        if "randomized" in abs_text.lower():
            appraisal.append({
                "id": i, "title": art['title'], "type": "RCT",
                "Jadad": auto_jadad_score(abs_text),
                "GRADE": auto_grade(abs_text)
            })
        elif "cohort" in abs_text.lower():
            nos_detail, nos_total = auto_nos_score(abs_text)
            appraisal.append({
                "id": i, "title": art['title'], "type": "Cohort",
                "NOS": nos_detail, "NOS_Total": nos_total,
                "GRADE": auto_grade(abs_text)
            })
        else:
            appraisal.append({
                "id": i, "title": art['title'], "type": "Other",
                "GRADE": auto_grade(abs_text)
            })
    return appraisal

# -------- Step 4. One-click Report Generator -------- #
def generate_ebm_report(patient, intervention, comparison, outcomes, max_results=10, filename="EBM_Report.docx"):
    query = generate_pico_queries(patient, intervention, comparison, outcomes)
    print("🔍 PubMed 查詢式:\n", query, "\n")

    articles = fetch_pubmed_articles(query, max_results=max_results)
    appraisal = auto_appraise(articles)

    doc = Document()
    doc.add_heading("EBM Clinical Research Report", 0)

    # Title (PICO Summary)
    doc.add_heading("PICO Question", level=1)
    doc.add_paragraph(f"Patient: {patient}")
    doc.add_paragraph(f"Intervention: {intervention}")
    doc.add_paragraph(f"Comparison: {comparison}")
    doc.add_paragraph(f"Outcome: {', '.join(outcomes)}")

    # Literature Search
    doc.add_heading("Literature Search", level=1)
    for i, art in enumerate(articles, 1):
        doc.add_paragraph(f"[{i}] {art['title']} ({art['journal']}, {art['year']})")
        doc.add_paragraph(f"Authors: {art['authors']}")
        doc.add_paragraph(f"Abstract: {art['abstract']}")
        doc.add_paragraph(f"PMID: {art['pmid']} | DOI: {art['doi']}\n")

    # Critical Appraisal
    doc.add_heading("Critical Appraisal", level=1)
    for a in appraisal:
        if a["type"] == "RCT":
            doc.add_paragraph(f"[{a['id']}] {a['title']} → RCT")
            doc.add_paragraph(f"  Jadad Score: {a['Jadad']}/5")
            doc.add_paragraph(f"  GRADE: {a['GRADE']}\n")
        elif a["type"] == "Cohort":
            doc.add_paragraph(f"[{a['id']}] {a['title']} → Cohort")
            doc.add_paragraph(f"  NOS: {a['NOS']} (Total {a['NOS_Total']}/9)")
            doc.add_paragraph(f"  GRADE: {a['GRADE']}\n")
        else:
            doc.add_paragraph(f"[{a['id']}] {a['title']} → Other")
            doc.add_paragraph(f"  GRADE: {a['GRADE']}\n")

    # Application Section
    doc.add_heading("Application", level=1)
    doc.add_paragraph("此部分可由研究者補充，包括：")
    doc.add_paragraph("- 臨床適用性（特定病人群體）")
    doc.add_paragraph("- 成本效益")
    doc.add_paragraph("- 對醫療品質與決策的影響")

    # References
    doc.add_heading("References (Vancouver)", level=1)
    for i, art in enumerate(articles, 1):
        ref = f"{i}. {art['authors']}. {art['title']}. {art['journal']}. {art['year']}; PMID:{art['pmid']}"
        if art["doi"]: ref += f". doi:{art['doi']}"
        doc.add_paragraph(ref)

    doc.save(filename)
    print(f"📄 已輸出報告：{filename}")
    return {"Articles": articles, "Appraisal": appraisal}
